# Proyecto Grupo 5 - Análisis Musical en Spotify

# Librerias y Config

In [2]:
import datetime
import typing

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

# spotify.sqlite

`spotify.sqlite` es el archivo con el que partimos este proyecto, obtenido de [los siguientes datos de kaggle](https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features). Contiene millones de canciones junto al analisis de caracteristicas que provee la api de spotify.

### Cargando datos

Estos se encuentran contenidos en una BDD `spotify.sqlite`, la cual tendremos en una carpeta `data`.

In [ ]:
import sqlite3

# funcion para cargar las tablas de la base de datos en un diccionario de dataframes
def load_data():
  # conexion a la base de datos
  conn = sqlite3.connect(
    database='data/spotify.sqlite'
  )
  # para que no haya problemas con los caracteres especiales
  conn.text_factory = lambda b: b.decode(encoding = 'utf-8', errors = 'ignore')
  
  # lista de tablas de la base de datos
  tables = [
    "r_albums_artists",
    "r_albums_tracks",
    "r_artist_genre",
    "r_track_artist",
    "genres",
    "albums",
    "artists",
    "audio_features",
    "tracks",
  ]

  # diccionario de dataframes
  dataframes : typing.Dict[str, pd.DataFrame] = {}
  
  # cargar cada tabla en un dataframe
  for table in tables:
    dataframes[table] = pd.read_sql_query(
      sql = f"SELECT * FROM {table}",
      con = conn
    )
  
  # cerrar la conexion a la base de datos
  conn.close()

  return dataframes

# ejecutar la funcion
dataframes : typing.Dict[str, pd.DataFrame] = load_data()
[
  tracks, artists, albums, genres, audio_features, 
  r_albums_artists, r_albums_tracks, r_artist_genre, r_track_artist
] = [
  dataframes['tracks'], dataframes['artists'], dataframes['albums'], dataframes['genres'], dataframes['audio_features'] ,
  dataframes['r_albums_artists'], dataframes['r_albums_tracks'], dataframes['r_artist_genre'], dataframes['r_track_artist']
]
# ver las tablas cargadas
dataframes.keys()

### Formato de las tablas

In [ ]:
info_df = pd.DataFrame()
info_df['tabla'] = dataframes.keys()
info_df['filas'] = [dataframes[i].shape[0] for i in dataframes.keys()]
info_df['columnas'] = [dataframes[i].shape[1] for i in dataframes.keys()]
info_df['nombre_columnas'] = [list(dataframes[i].columns) for i in dataframes.keys()]

info_df.set_index('tabla', inplace=True)
info_df

### Estadisticas y exploración inicial

Las tablas `r_albums_artists`, `r_albums_tracks`, `r_artist_genre` solamente contienen ids, por lo que no contienen estadisticas interesantes

#### albums

In [ ]:
albums.head()

In [ ]:
albums.shape

In [ ]:
albums.info()

In [ ]:
albums.describe()

En base a las celdas anteriores, consideramos: 

- Formalizar `release_date`, estan en formato timestamp unix, por lo que seria conveniente tener esa informacion como fecha. \
  Ademas debemos tener cuidado con los timestamps negativos, los que nos pueden traer problemas (algunos van al año 0)
- Analizar si la popularidad del album es importante, pues 75% de sus valores es 0 en un rango de 0 a 100


In [ ]:
albums['album_type'].value_counts()

In [ ]:
albums['album_group'].value_counts()

In [ ]:
(albums['album_group'] == '').value_counts()

En base a las celdas anteriores, consideramos: 

- `album_group`, que solo contiene strings vacios, es candidato a eliminar.

#### artists

In [ ]:
artists.head()

In [ ]:
artists.shape

In [ ]:
artists.info()

In [ ]:
artists.describe()

Aqui no consideramos modificar los datos. Pero hay que considerar un analisis mas en profundidad de las columnas de popularidad y seguidores.

#### tracks

In [ ]:
tracks.head()

In [ ]:
tracks.shape

In [ ]:
tracks.info()

In [ ]:
tracks.describe()

A partir de las celdas anteriores, consideramos:

- `disc_number` es candidato a eliminar, pues tiene poca varianza y algunos outliers.
- Analizar si `track_number` puede ser una columna util o no. Parece tener outliers (ver `max`).
- `is_playable` es candidato a eliminar, pues su count indica que muchos de sus valores son nulos.

In [ ]:
tracks['disc_number'].value_counts()

In [ ]:
tracks['disc_number'].max()

In [ ]:
tracks['disc_number'].var()

In [ ]:
tracks['track_number'].value_counts()

OPAAA, que paso ahi?
investiguemos 😎

In [ ]:
tracks['track_number'].max()

In [ ]:
tracks[tracks['track_number'] >= 1000].sample(5)

Curioso, estas canciones estan relacionadas y tienen su numero de **_Chapter_** como `track_number`, esto nos genera una serie de outliers que posiblemente van de track number 1 a 1522. Extendamos la busqueda.

In [ ]:
# obtenemos todos los tracks del album que contiene el track con el numero mas alto
r_albums_tracks[
  r_albums_tracks['album_id'] == r_albums_tracks[
    # album que tiene el track con el numero mas alto
    r_albums_tracks['track_id'] == tracks[
      # track con el numero mas alto
      tracks['track_number'] == tracks['track_number'].max()
    ]['id'].values[0]
  ]['album_id'].values[0]
].shape

Exactamente lo que pensabamos, tenemos nuestro primer outlier. Utilizemos `track_number` como filtro para ver si encontramos otros.

In [ ]:
tracks[tracks['track_number'] >= 500].sample(5)

En efecto, Aparece nuevamente el conde de montecristo, pero ademas comenzamos a encontrar audios de la biblia (?), efectos de sonido y canciones de musica clasica (posiblemente de albumes recopilatorios).

En base a esto podemos definir que `track_number` nos puede servir como filtro a todos estos datos que no nos sean utiles y que quizas podemos excluir los albumes recopilatorios de los datos.

Finalmente, hay que decidir donde pondremos nuestro limite a considerar y si excluimos los albumes recopilatorios de los datos.

In [ ]:
tracks['is_playable'].value_counts()

Ya habiamos visto que a esta columna le faltaban muchos datos, comprobemos cuantos son nulos.

In [ ]:
tracks[tracks['is_playable'].isna()].shape

Hay muchos nulos en esta columna!!! Claramente es eliminable.

In [ ]:
tracks['explicit'].value_counts()

In [ ]:
tracks['duration'].value_counts()

La duracion tambien evidencia presencia de algunos outliers, sin embargo, conviene excluir a los albumes recopilatorios del analisis y verificar nuevamente.

#### audio_features

In [ ]:
audio_features.head()

In [ ]:
audio_features.shape

In [ ]:
audio_features.info()

In [ ]:
audio_features.describe()

Aqui solo eliminaremos `analysis_url` (no nos sirve sin un token), pues es la tabla con la informacion de mayor interes.

#### genres

In [ ]:
genres.head()

In [ ]:
genres.shape

In [ ]:
genres.info()

In [ ]:
genres.value_counts()

Esta tabla va relacionada al artista, no a las canciones o albumes, por lo que hay que considerar si es realmente util o no.

### Eliminar y modificar columnas

#### albums

Inicialmente, modificaremos `release_date` para que sea en formato de fecha y no timestamp, luego eliminamos `album_group` debido a que todos son valores vacios.

In [ ]:
# debido a limitaciones en los timestamps de pandas, utilizamos la clase datetime de python (la columna será de tipo object para pandas)
# no tan fun fact: pd.Timestamp.min es el 1 de enero de 1677 :(

# Para rematar, fromtimestamp() no acepta fechas negativas, por lo que tenemos que hacer un workaround con timedelta

# funcion para convertir un timestamp a una fecha
def convert_timestamp_to_date(timestamp):
  try:
    # pasamos el timestamp a dias y luego a fecha
    return (datetime.datetime.fromtimestamp(0) + datetime.timedelta(days = timestamp / 1000 / 60 / 60 / 24)).date()
  # existe un limite para las fechas negativas, lo que nos da un error de OverflowError
  except OverflowError:
    # si el timestamp es positivo, devolvemos la fecha maxima
    if timestamp > 0:
      return datetime.datetime.max.date()
    # si el timestamp es negativo, devolvemos la fecha minima
    else:
      return datetime.datetime.min.date()

In [ ]:
# pasar release_date de timestamp a datetime
albums['album_release_date'] = albums['release_date'].apply(
  lambda x: convert_timestamp_to_date(x)
)
albums['album_release_year'] = albums['album_release_date'].apply(
  lambda x: x.year
)
albums['album_release_month'] = albums['album_release_date'].apply(
  lambda x: x.month
)
# OJO: probablemente no sea muy preciso
albums['album_release_day'] = albums['album_release_date'].apply(
  lambda x: x.day
)

albums.drop(columns=[
  'album_group', 'release_date',
], inplace=True)
albums.rename(columns={
  'id': 'album_id',
  'name': 'album_name',
  'popularity': 'album_popularity',
}, inplace=True)
albums.head()

In [ ]:
albums.describe().apply(lambda s: s.apply(lambda x: format(x, '.0f')))

#### artists

In [ ]:
artists.rename(columns={
  'id': 'artist_id',
  'name': 'artist_name',
  'popularity': 'artist_popularity',
  'followers': 'artist_followers',
}, inplace=True)
artists.head()

#### tracks

Inicialmente, consideramos eliminar `disc_number` por baja varianza y `is_playable` por muchos valores nulos.
Ademas cambiaremos algunos nombres de columnas.

In [ ]:
tracks.drop(columns=[
  'preview_url', 'disc_number', 'is_playable'
], inplace=True)
tracks.rename(columns={
  'id': 'track_id',
  'name': 'track_name',
  'duration': 'track_duration_ms',
  'track_number': 'track_number_in_album',
  'explicit': 'track_explicit',
  'popularity': 'track_popularity'
}, inplace=True)
tracks.head()

#### audio_features

In [ ]:
audio_features.drop(columns=[
  'analysis_url',
], inplace=True)
audio_features.rename(columns={
  'id': 'audio_feature_id',
  'duration_ms': 'feature_duration_ms',
}, inplace=True)
audio_features.head()

### Mergear tablas

In [ ]:
merged_data = pd.merge(
  left = r_albums_tracks,
  right = tracks,
  how = 'inner',
  on = 'track_id',
)

merged_data = pd.merge(
  left = merged_data,
  right = albums,
  how = 'inner',
  on = 'album_id',
)

merged_data = pd.merge(
  left = r_albums_artists,
  right = merged_data,
  how = 'inner',
  on = 'album_id',
)

merged_data = pd.merge(
  left = merged_data,
  right = artists,
  how = 'inner',
  on = 'artist_id',
)

merged_data = pd.merge(
  left = merged_data,
  right = audio_features,
  how = 'inner',
  on = 'audio_feature_id',
)

In [ ]:
merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
merged_data.info()

### Limitar/Filtrar data mergeada

Dentro de las modificaciones para el hito 2, consideramos limitar el periodo, la duracion y eliminar los albumes recopilatorios. \
Además, aqui tambien eliminaremos completamente los albumes que posean más de 30 canciones, eliminando uno de los casos que vimos durante la exploración.

In [ ]:
MIN_YEAR_TO_CONSIDER = 2000
MAX_YEAR_TO_CONSIDER = 2019
MIN_TRACK_DURATION = 1 * 60 * 1000 # 1 minuto como minimo
MAX_TRACK_DURATION = 6 * 60 * 1000 # 6 minutos como maximo
MAX_TRACK_NUMBER_IN_ALBUM = 30 # 30 canciones como maximo en un album (si no se cumple, se eliminan todas las canciones del album)
DELETED_TYPES = ['compilation']

merged_data = merged_data[
  (merged_data['album_release_year'] >= MIN_YEAR_TO_CONSIDER) &
  (merged_data['album_release_year'] <= MAX_YEAR_TO_CONSIDER) &
  (merged_data['track_duration_ms'] >= MIN_TRACK_DURATION) &
  (merged_data['track_duration_ms'] <= MAX_TRACK_DURATION) &
  (~merged_data['album_type'].isin(DELETED_TYPES))
]

removable_albums_ids = merged_data[
  merged_data['track_number_in_album'] > MAX_TRACK_NUMBER_IN_ALBUM
]['album_id'].unique()

merged_data = merged_data[
  ~merged_data['album_id'].isin(removable_albums_ids)
]

merged_data.drop_duplicates(subset=[
  'track_id', 'artist_id', 'album_id'
], inplace=True)


In [ ]:
merged_data.info()

### Guardado de datos en un csv

In [ ]:
# cambiar estos valores para guardar el dataframe
SAVE = False
if SAVE:
  merged_data.to_csv('data/merged_data.csv', index=False)

## Graficando

In [ ]:
# tomamos una muestra de los datos
fraction = 0.4
sampled_data = merged_data.sample(frac=fraction, random_state=42)
sampled_data.shape

In [ ]:
sampled_data['track_duration_s'] = sampled_data['track_duration_ms'] / 1000

In [ ]:
# histograma de la duracion de las canciones
sns.histplot(
  data = sampled_data,
  x = 'track_duration_s',
  bins = 100,
  kde = True,
)

In [ ]:
sns.histplot(
  data = sampled_data,
  x = 'track_popularity',
  bins = 25,
  kde = True,
)


In [ ]:
# plot release year
sns.histplot(
  data = sampled_data,
  x = 'album_release_year',
  bins = 20,
)

In [ ]:

# plot multiples histogramas (13 columnas/features)
fig, axs = plt.subplots(4, 4, figsize=(20, 20))

columns = sampled_data.columns[-13:]
for i, col in enumerate(columns):
  sns.histplot(
    data = sampled_data,
    x = col,
    ax = axs[i // 4][i % 4],
    bins = 25,
    kde = True,
  )


# merged_data.csv

`merged_data.csv` es el archivo resultante del procesamiento hecho en la seccion de `spotify.sqlite`, contiene todos los datos que consideramos luego de limpieza y seleccion de sus registros en un solo dataframe.

## carga

In [5]:
merged_data = pd.read_csv('data/merged_data.csv')
merged_data.head()

,album_id,artist_id,track_id,track_duration_ms,track_explicit,audio_feature_id,track_name,track_number_in_album,track_popularity,album_name,album_type,album_popularity,album_release_date,album_release_year,album_release_month,album_release_day,artist_name,artist_popularity,artist_followers,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,1GZik94t53uY8oIANFq002,3VBpsrUi2vV7Uj87ONHu7Z,2XpZPnrvDqpKcNcfv6fviu,208759,0,2XpZPnrvDqpKcNcfv6fviu,I Put A Spell On You,1,28,My Little Shop Of Horrors,album,29,2006-08-16,2006,8,16,Screamin' Jay Hawkins,45,75914,0.3090,0.6010,208760,0.5990,0.0007,5,0.8230,-12.3040,0,0.2630,133.0850,3,0.6750
1,1GZik94t53uY8oIANFq002,3VBpsrUi2vV7Uj87ONHu7Z,1GZik94t53uY8oIANFq002,254879,0,1GZik94t53uY8oIANFq002,Portrait Of A Man,2,25,My Little Shop Of Horrors,album,29,2006-08-16,2006,8,16,Screamin' Jay Hawkins,45,75914,0.5730,0.4460,254880,0.2080,0.0167,1,0.1270,-14.1970,1,0.0325,145.9490,3,0.2780
2,1GZik94t53uY8oIANFq002,3VBpsrUi2vV7Uj87ONHu7Z,6xhWtLM0DoHRbf7kbU9ZID,297879,0,6xhWtLM0DoHRbf7kbU9ZID,What's Gonna Happen On The 8th Day,3,4,My Little Shop Of Horrors,album,29,2006-08-16,2006,8,16,Screamin' Jay Hawkins,45,75914,0.6410,0.5780,297880,0.5070,0.0000,10,0.3330,-11.6940,1,0.0895,114.6480,3,0.5140
3,1GZik94t53uY8oIANFq002,3VBpsrUi2vV7Uj87ONHu7Z,2R1Eime7xs6QeBmbzWW2qH,168519,0,2R1Eime7xs6QeBmbzWW2qH,We Love,4,3,My Little Shop Of Horrors,album,29,2006-08-16,2006,8,16,Screamin' Jay Hawkins,45,75914,0.5720,0.3730,168520,0.4020,0.0000,0,0.1280,-12.6550,1,0.0427,174.1390,3,0.4010
4,1GZik94t53uY8oIANFq002,3VBpsrUi2vV7Uj87ONHu7Z,5FvPZyLxWlBqGbZmzDW2Wr,226092,0,5FvPZyLxWlBqGbZmzDW2Wr,Please Don't Leave Me,5,3,My Little Shop Of Horrors,album,29,2006-08-16,2006,8,16,Screamin' Jay Hawkins,45,75914,0.2850,0.4210,226093,0.8050,0.0008,0,0.1580,-11.4710,1,0.0997,170.7830,4,0.7570


In [6]:
merged_data.shape

(4508534, 32)

In [7]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4508534 entries, 0 to 4508533
Data columns (total 32 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   album_id               object 
 1   artist_id              object 
 2   track_id               object 
 3   track_duration_ms      int64  
 4   track_explicit         int64  
 5   audio_feature_id       object 
 6   track_name             object 
 7   track_number_in_album  int64  
 8   track_popularity       int64  
 9   album_name             object 
 10  album_type             object 
 11  album_popularity       int64  
 12  album_release_date     object 
 13  album_release_year     int64  
 14  album_release_month    int64  
 15  album_release_day      int64  
 16  artist_name            object 
 17  artist_popularity      int64  
 18  artist_followers       int64  
 19  acousticness           float64
 20  danceability           float64
 21  duration               int64  
 22  energy            

# songs_normalize.csv

`songs_normalize.csv` es [un dataset de kaggle](https://www.kaggle.com/datasets/paradisejoy/top-hits-spotify-from-20002019) de los mejores hits de spotify, considerando desde 2000 al 2019.

## carga y modificaciones iniciales

In [ ]:
songs_normalize = pd.read_csv('data/songs_normalize.csv')
songs_normalize.head()

In [ ]:
songs_normalize.shape

2000 filas y 18 columnas

In [ ]:
songs_normalize.info()

Podemos tratar un poco la columna `genre`, para que sea una lista de generos para cada cancion

In [ ]:
songs_normalize['genre'] = songs_normalize['genre'].apply(
  # split and then strip each genre
  lambda x: [i.strip() for i in x.split(',')]
)
songs_normalize['genre'].head()

## analisis y tratamiento de nulos

In [ ]:
print(f"=== Has NaN? ===")
for col in songs_normalize.columns:
  if col == 'genre':
    print(col, songs_normalize[col].apply(lambda x: len(x) == 0).any())
  else:
    print(col, songs_normalize[col].hasnans)

## analisis de columnas

In [ ]:
songs_normalize.describe()

In [ ]:
songs_normalize.info()

In [ ]:
songs_normalize['explicit'].value_counts()

In [ ]:
songs_normalize['year'].value_counts().sort_index()

Tenemos un buen balance de canciones que salieron en los 2000 - 2019, pero fuera de ese rango existe una cantidad significativa de canciones.

In [ ]:
# obtain the genres
songs_normalize_genres = dict()
for genre_list in songs_normalize['genre']:
  for genre in genre_list:
    if genre not in songs_normalize_genres:
      songs_normalize_genres[genre] = 0
    songs_normalize_genres[genre] += 1

songs_normalize_genres = pd.DataFrame.from_dict(songs_normalize_genres, orient='index', columns=['count'])
songs_normalize_genres.sort_values(by='count', ascending=False, inplace=True)
songs_normalize_genres.head()

In [ ]:
songs_normalize_genres.shape

Solamente hay 15 generos

# charts.csv

`charts.csv` es [un dataset de kaggle](https://www.kaggle.com/datasets/dhruvildave/spotify-charts) que compila las colecciones "Top 200" y "50 virales" desde 2017 al 2021. Estas colecciones se publican globalmente cada 2-3 dias, dando informacion valiosa historica y alto valor de analisis.

## carga y modificaciones iniciales

In [ ]:
charts = pd.read_csv('data/charts.csv')
charts.head()

In [ ]:
charts.shape

26.173.541 filas y 9 columnas

In [ ]:
charts.info()

Inicialmente notemos que hay columnas que podemos expresar como de tipo fechas y otras declarar como columnas categoricas.

In [ ]:
charts['date'] = pd.to_datetime(charts['date'])
charts['region'] = charts['region'].astype('category')
charts['chart'] = charts['chart'].astype('category')
charts['trend'] = charts['trend'].astype('category')

In [ ]:
charts.info()

## analisis y tratamiento de nulos

In [ ]:
print(f"=== Has NaN? ===")
for col in charts.columns:
  print(col, charts[col].hasnans)

In [ ]:
charts['title'].hasnans

`title` tiene valores faltantes, veamos cuantos son

In [ ]:
charts[charts['title'].isna()].shape

solamente 11 filas con titulo de cancion faltante veamos cuales son

In [ ]:
charts[charts['title'].isna()]

Accediendo a la url y al artista por internet, podemos encontrar que existe una cancion llamada "NA", probablemente ocurrio algun error al ser cargadas. Podemos arreglarlo facilmente.

In [ ]:
# fix the title "NA"
charts.loc[charts['title'].isna(), 'title'] = 'NA'

In [ ]:
charts[charts['title'].isna()].shape

In [ ]:
charts[charts['title']=='NA']

In [ ]:
charts['artist'].hasnans

`artist` tiene valores faltantes, veamos cuantos son

In [ ]:
charts[charts['artist'].isna()].shape

solamente 18 filas tienen artista faltante, veamos cuales son

In [ ]:
charts[charts['artist'].isna()]

Buscando en internet la cancion, podemos identificar que el artista tiene nombre "N/A", probablemente ocurrio algun error al ser cargadas. Solucionemoslo.

In [ ]:
# fix the artist name "N/A"
charts.loc[charts['artist'].isna(), 'artist'] = 'N/A'

In [ ]:
charts[charts['artist'].isna()].shape

In [ ]:
charts[charts['artist']=='N/A']

In [ ]:
charts['streams'].hasnans

`streams` tiene valores faltantes, veamos cuantos son

In [ ]:
charts[charts['streams'].isna()].shape

5.851.610 filas tienen `streams` faltantes, veamos a que colecciones pertenecen

In [ ]:
charts[charts['streams'].isna()]['chart'].value_counts()

Todas las 5.851.610 filas son de la coleccion "50 Virales", comprobemos si es que esa coleccion no tiene `streams` en todos sus casos.

In [ ]:
charts[charts['chart']=='viral50']['streams'].isna().value_counts()

En efecto, las 5.851.610 filas que no poseen `streams` es debido a que las canciones de la coleccion "50 Virales" no poseen tal informacion.

## analisis de columnas

In [ ]:
charts.describe()

In [ ]:
charts['region'].value_counts().sort_values(ascending=False)

In [ ]:
charts['chart'].value_counts()

Los datos se encuentran en proporcion 200:50 = 4:1

In [ ]:
charts['trend'].value_counts()

In [ ]:
charts['title'].unique().shape, charts['artist'].unique().shape

Tenemos 164.807 canciones y 96.157 artistas unicos

In [ ]:
charts['date'].unique().shape

tenemos 1.826 fechas distintas

In [ ]:
charts['date'].dt.year.value_counts().sort_index()

En general, tenemos datos balanceados en los años

# data.csv

`data.csv` es un [dataset de kaggle](https://www.kaggle.com/datasets/ivannatarov/spotify-daily-top-200-songs-with-genres-20172021) que contiene la coleccion "Top 200" obtenida diariamente desde 2017 hasta 2021. \
El usuario que subio este dataset declara que es util para principiantes, dado que contiene cosas que se suelen tratar en la exploracion de datos.

## carga y modificaciones iniciales

In [52]:
data = pd.read_csv('data/data.csv', delimiter='#')
data.head()

,Position,Track Name,Artist,Streams,Date,Genre
0,1,Starboy,The Weeknd,3135625,2017-01-01,"['canadian pop', 'canadian contemporary r&b', 'pop']"
1,2,Closer,The Chainsmokers,3015525,2017-01-01,"['pop', 'pop dance', 'tropical house', 'edm', 'electropop', 'dance pop']"
2,3,Let Me Love You,DJ Snake,2545384,2017-01-01,"['pop', 'electronic trap', 'dance pop', 'edm', 'pop dance', 'pop rap']"
3,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,2017-01-01,"['pop', 'uk dance', 'dance pop', 'uk funky', 'tropical house', 'pop dance', 'post-teen pop', 'edm']"
4,5,One Dance,Drake,2259887,2017-01-01,"['toronto rap', 'canadian pop', 'canadian hip hop', 'rap', 'pop rap', 'hip hop']"


In [53]:
data.shape

(321200, 6)

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321200 entries, 0 to 321199
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Position    321200 non-null  int64 
 1   Track Name  321182 non-null  object
 2   Artist      321182 non-null  object
 3   Streams     321200 non-null  int64 
 4   Date        321200 non-null  object
 5   Genre       321182 non-null  object
dtypes: int64(2), object(4)
memory usage: 14.7+ MB


In [55]:
data['Date'] = pd.to_datetime(data['Date'])

los generos estan como string para pandas, pero podemos notar que tiene nulos, tenemos que tratarlos primero

## analisis y tratamiento de nulos

In [56]:
print(f"=== Has NaN? ===")
for col in data.columns:
  print(col, data[col].hasnans)

=== Has NaN? ===
Position False
Track Name True
Artist True
Streams False
Date False
Genre True


In [57]:
data[data['Track Name'].isna()].shape

(18, 6)

In [58]:
data[data['Artist'].isna()].shape

(18, 6)

In [59]:
data[data['Genre'].isna()].shape

(18, 6)

es bastante sospechoso que sean 18 en todas estas columnas

In [60]:
data[data['Track Name'].isna() | data['Artist'].isna() | data['Genre'].isna()]

,Position,Track Name,Artist,Streams,Date,Genre
39204,5,NaN,NaN,3568811,2017-07-20,NaN
39212,13,NaN,NaN,2571960,2017-07-20,NaN
39229,30,NaN,NaN,1798208,2017-07-20,NaN
39238,39,NaN,NaN,1520291,2017-07-20,NaN
39404,5,NaN,NaN,3653533,2017-07-21,NaN
39415,16,NaN,NaN,2522453,2017-07-21,NaN
39434,35,NaN,NaN,1798890,2017-07-21,NaN
39447,48,NaN,NaN,1526955,2017-07-21,NaN
39700,101,NaN,NaN,747893,2017-07-22,NaN
39888,89,NaN,NaN,690247,2017-07-23,NaN


Lamentablemente no hay muchos datos que podamos sacar de esto sin las columnas que faltan

In [61]:
remove_data = data[data['Track Name'].isna() | data['Artist'].isna() | data['Genre'].isna()]
data.drop(remove_data.index, inplace=True)
del remove_data

print(f"=== Has NaN? ===")
for col in data.columns:
  print(col, data[col].hasnans)

=== Has NaN? ===
Position False
Track Name False
Artist False
Streams False
Date False
Genre False


Ahora que ya no hay nulos, trataremos la columna `Genre`

In [62]:
import re
def process_genres_str(x: str):
  # delete ' and " characters
  # delete the first and last character (they are [ and ])
  # split by , and then strip each genre
  return [i.strip() for i in re.sub(r'\'|\"', '', x)[1:-1].split(',')]

data['Genre'] = data['Genre'].apply(process_genres_str)
data['Genre'].head()

0                                         [canadian pop, canadian contemporary r&b, pop]
1                           [pop, pop dance, tropical house, edm, electropop, dance pop]
2                             [pop, electronic trap, dance pop, edm, pop dance, pop rap]
3    [pop, uk dance, dance pop, uk funky, tropical house, pop dance, post-teen pop, edm]
4                   [toronto rap, canadian pop, canadian hip hop, rap, pop rap, hip hop]
Name: Genre, dtype: object

## analisis de columnas

In [66]:
data.describe()

,Position,Streams,Date
count,321182.0000,321182.0000,321182
mean,100.5028,1188463.9883,2019-03-19 19:07:35.405346816
min,1.0000,325951.0000,2017-01-01 00:00:00
25%,51.0000,700483.2500,2018-02-10 00:00:00
50%,101.0000,895351.0000,2019-03-19 00:00:00
75%,151.0000,1364059.0000,2020-04-23 00:00:00
max,200.0000,17223237.0000,2021-07-17 00:00:00
std,57.7336,839659.5540,NaN


In [67]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 321182 entries, 0 to 321199
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Position    321182 non-null  int64         
 1   Track Name  321182 non-null  object        
 2   Artist      321182 non-null  object        
 3   Streams     321182 non-null  int64         
 4   Date        321182 non-null  datetime64[ns]
 5   Genre       321182 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 17.2+ MB


In [68]:
data['Track Name'].unique().shape, data['Artist'].unique().shape

((5496,), (1127,))

5.496 canciones y 1.127 artistas unicos

In [69]:
data['Date'].dt.year.value_counts().sort_index()

Date
2017    72182
2018    73000
2019    73000
2020    72000
2021    31000
Name: count, dtype: int64

Tenemos un muy buen balance entre las canciones de 2017 - 2020, en 2021 faltan, pero es esperable dada la fecha maxima encontrada

In [64]:
# obtain the genres
data_genres = dict()
for genre_list in data['Genre']:
  for genre in genre_list:
    if genre not in data_genres:
      data_genres[genre] = 0
    data_genres[genre] += 1

data_genres = pd.DataFrame.from_dict(data_genres, orient='index', columns=['count'])
data_genres.sort_values(by='count', ascending=False, inplace=True)
data_genres.head()

,count
pop,146708
dance pop,89558
rap,66629
post-teen pop,64118
pop rap,48831


In [65]:
data_genres.shape

(636, 1)

tenemos 636 generos